In [18]:
import pandas as pd
import numpy as np

import missingno

In [26]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [23]:
df_train.head(3)

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,nacionalidade,estado_onde_nasceu,estado_onde_reside,possui_telefone_residencial,codigo_area_telefone_residencial,tipo_residencia,meses_na_residencia,possui_telefone_celular,possui_email,renda_mensal_regular,renda_extra,possui_cartao_visa,possui_cartao_mastercard,possui_cartao_diners,possui_cartao_amex,possui_outros_cartoes,qtde_contas_bancarias,qtde_contas_bancarias_especiais,valor_patrimonio_pessoal,possui_carro,vinculo_formal_com_empresa,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,1,CE,CE,Y,107,1.0,12.0,N,0,480.0,0.0,0,0,0,0,0,1,1,0.0,1,N,,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,1,SE,SE,Y,91,1.0,5.0,N,1,380.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,1,BA,BA,Y,90,5.0,1.0,N,1,600.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1


In [27]:
df_train.drop(['id_solicitante', 'estado_onde_nasceu', 'codigo_area_telefone_residencial', 'possui_telefone_trabalho', 'codigo_area_telefone_trabalho'], axis=1, inplace=True)
df_test.drop(['id_solicitante', 'estado_onde_nasceu', 'codigo_area_telefone_residencial', 'possui_telefone_trabalho', 'codigo_area_telefone_trabalho'], axis=1, inplace=True)

In [ ]:
df_train['sexo'] = df_train['sexo'].map({'M': 1, 'F': 0})
df_train['sexo'] = df_train['sexo'].map({'M': 1, 'F': 0})



In [20]:
df_train.isna().sum()

id_solicitante                          0
produto_solicitado                      0
dia_vencimento                          0
forma_envio_solicitacao                 0
tipo_endereco                           0
sexo                                    0
idade                                   0
estado_civil                            0
qtde_dependentes                        0
grau_instrucao                          0
nacionalidade                           0
estado_onde_nasceu                      0
estado_onde_reside                      0
possui_telefone_residencial             0
codigo_area_telefone_residencial        0
tipo_residencia                       536
meses_na_residencia                  1450
possui_telefone_celular                 0
possui_email                            0
renda_mensal_regular                    0
renda_extra                             0
possui_cartao_visa                      0
possui_cartao_mastercard                0
possui_cartao_diners              

In [22]:
df_test.isna().sum()

id_solicitante                         0
produto_solicitado                     0
dia_vencimento                         0
forma_envio_solicitacao                0
tipo_endereco                          0
sexo                                   0
idade                                  0
estado_civil                           0
qtde_dependentes                       0
grau_instrucao                         0
nacionalidade                          0
estado_onde_nasceu                     0
estado_onde_reside                     0
possui_telefone_residencial            0
codigo_area_telefone_residencial       0
tipo_residencia                      125
meses_na_residencia                  362
possui_telefone_celular                0
possui_email                           0
renda_mensal_regular                   0
renda_extra                            0
possui_cartao_visa                     0
possui_cartao_mastercard               0
possui_cartao_diners                   0
possui_cartao_am

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
X_train[:, 24:30] = imputer.fit_transform(X_train[:, 24:30])

imputer_kaggle = SimpleImputer(missing_values=np.nan, strategy="mean")
X_kaggle[:, 24:30] = imputer_kaggle.fit_transform(X_kaggle[:, 24:30])
